In [1]:
import pandas as pd
import requests
import logging
from datetime import datetime, timedelta

In [2]:
exist_df = pd.read_csv(r".\\BITSTAMP_BTCUSD_1D.csv")
exist_df = exist_df[["time", "open", "high", "low", "close"]]

In [3]:
start = exist_df.iloc[-1, 0]
end = datetime.today().date()
dates = pd.date_range(start, end)
dates = [int(x.value/(10**9)) for x in dates]
df = pd.DataFrame(index = dates, columns =  [["close", "high", "low", "open", "timestamp", "volume"]])

In [4]:
# Initialize logging
logging.basicConfig(level=logging.INFO)

def request_bitstamp_data(pair="btcusd", limit=500, start_date=start, end_date=end):
    """
    Request Bitstamp API to get OHLC data and return it as a list of dictionaries.

    Parameters:
    pair (str): Currency pair to fetch data for. Default is "btcusd".
    limit (int): Maximum number of data points (days) to fetch in each API call. Default is 100.
    start_date (str): The start date for data fetching in 'YYYY-MM-DD' format. Default is "2023-01-01".
    end_date (str): The end date for data fetching in 'YYYY-MM-DD' format. Default is "2023-02-01".

    Returns:
    list: A list of dictionaries containing OHLC data.
    """
    data = []
    
    # Generate the list of dates
    date_format = "%Y-%m-%d"
    start_date = datetime.strptime(start_date, date_format).date()
    
    
    current_start_date = start_date
    
    while current_start_date < end_date:
        current_end_date = min(current_start_date + timedelta(days=limit), end_date)
        
        parameters = {
            "step": 86400,  # 1 day in seconds
            "limit": limit,
            "start": int(datetime(start_date.year, start_date.month, start_date.day).timestamp()),
            "end": int(datetime(end_date.year, end_date.month, end_date.day).timestamp())
        }

        logging.info(f"Requesting data from {current_start_date} to {current_end_date}")
        
        try:
            response = requests.get(f"https://www.bitstamp.net/api/v2/ohlc/{pair}/", params=parameters)
            response.raise_for_status()  # Raise an exception for HTTP errors
            req = response.json()["data"]["ohlc"]
            if not req:
                logging.warning(f"No data returned for period {current_start_date.date()} to {current_end_date.date()}")
            data += req
        except requests.RequestException as e:
            logging.error(f"Request failed: {e}")
            break
        
        # Move to the next date range
        current_start_date = current_end_date + timedelta(days=1)
    
    logging.info(f"Data download for pair: {pair.upper()} completed")
    return data

In [5]:
# Executing request and gethering data
ohlc_data = request_bitstamp_data(pair="btcusd")


INFO:root:Requesting data from 2024-07-09 to 2024-07-10
INFO:root:Data download for pair: BTCUSD completed


In [8]:
df = pd.DataFrame(ohlc_data)
df = df.drop_duplicates()
df = df.reset_index()
df = df.drop(labels = ["index"], axis = 1)

if df.empty:
    print(f"No data add. Existing dataframe is up to date.")
    exist_df.to_csv("btcusd_data.csv", sep=",", columns = ["timestamp", "open", "high", "low","close"])
else:
    df.to_csv("btcusd_data.csv", sep=",", columns = ["timestamp", "open", "high", "low","close"])